In [1]:
import pandas as pd

import numpy as np

import statsmodels.formula.api as smf

from scipy.stats import norm

from datetime import date

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
import plotly.express as px

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [2]:
# Shelter in Place Dates
state_county_lockdown = pd.read_csv('../../data/nytimes/county-shelter-in-place.csv')
# County Population
state_county_pop = pd.read_csv('../../data/census/co-est2019-alldata.csv', encoding = 'latin-1')
census_var = 'CENSUS2010POP'

In [3]:
# National Figure Plot
nat_lockdown = pd.merge(state_county_lockdown,state_county_pop[['STATE','COUNTY',census_var]],
                        on=['STATE','COUNTY'])
nat_lockdown.loc[nat_lockdown.SHELTER_DATE.isnull(),'SHELTER_DATE'] = '12/31/2020'
nat_lockdown['SHELTER_dt'] = pd.to_datetime(nat_lockdown.SHELTER_DATE)
nat_lockdown = pd.DataFrame(nat_lockdown.groupby('SHELTER_dt')[census_var].sum())
nat_lockdown['cumulative_pop'] = nat_lockdown[census_var].cumsum()
nat_lockdown['pop_share_lock'] = nat_lockdown.cumulative_pop/nat_lockdown.loc['12/31/2020']['cumulative_pop']
nat_lockdown = nat_lockdown[:'04-05-2020']

idx = pd.date_range('03-14-2020', '04-05-2020')

nat_lockdown = nat_lockdown.reindex(idx)
nat_lockdown.fillna(method='ffill',inplace=True)
nat_lockdown.fillna(0,inplace=True)

In [4]:
fig_nat = go.Figure(data=[go.Scatter(y=nat_lockdown[:date.today()]['pop_share_lock'], 
                                    x=nat_lockdown[:date.today()].index,line=dict(color="black"),
                                    textfont=dict(color="black"))])

fig_nat.update_xaxes(title_text='',tickfont=dict(color='black'))
fig_nat.update_yaxes(title_text='Share of Population Under Stay-at-Home Order',title_font=dict(color='black'),tickfont=dict(color='black'))
fig_nat.update_layout(margin = dict(l=20, r=20, t=0, b=20))


# Create scatter trace of text labels
fig_nat.add_trace(go.Scatter(
    x=['2020-03-17', '2020-03-24','2020-04-01'],
    y=[.55, .15, .65],
    text=["Week Ending March 21","Week Ending March 28", "Week Ending April 4"],
    mode="text",
    textfont=dict(
        color="black"
    )
))

fig_nat.update_layout(showlegend=False,
    shapes=[
        # 1st highlight during Feb 4 - Feb 6
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0="2020-03-14",
            y0=-.001,
            x1="2020-03-21",
            y1=1,
            fillcolor="White",
            opacity=0.5,
            layer="below",
            line_width=0,
        ),
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0="2020-03-21",
            y0=-.001,
            x1="2020-03-28",
            y1=1,
            fillcolor="LightGray",
            opacity=0.5,
            layer="below",
            line_width=0,
        ),
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0="2020-03-28",
            y0=-.001,
            x1="2020-04-04",
            y1=1,
            fillcolor="Gray",
            opacity=0.3,
            layer="below",
            line_width=0,
        )
    ]
)

fig_nat.show()

fig_nat.write_image("../../output/plots/nat-SAH-share.pdf", 
                    width=900, height=400, scale=3)

In [5]:
state_df = pd.read_stata('../../data/stata/state_build.dta')
# State Stay at Home thru April 4
apr4_idx = state_df.daten == '2020-04-04'

In [6]:
fig_state_shares = go.Figure()

fig_state_shares.add_trace(go.Bar(x=state_df[apr4_idx]['state_name'],
                y=state_df[apr4_idx]['SAH_cumexp'],
                name='Employment Weighted Duration of Stay-at-Home (in Weeks)',
                marker_color='black',
                textfont=dict(color="black")
                ))

fig_state_shares.update_layout(
    legend=dict(
        x=.25,
        y=1.0,
        bgcolor='LightGray',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    plot_bgcolor='LightGray'
)
fig_state_shares.update_xaxes(tickfont=dict(color='black'))
fig_state_shares.update_yaxes(tickfont=dict(color='black'))

fig_state_shares.show()

fig_state_shares.write_image("../../output/plots/state-emp-shares.pdf", width=900, height=600, scale=3)

In [7]:
state_df['<b>Conf. C19 <br>per 1K p.c.</b>'] = state_df['confirm_to_1k']
fig_emp_scatt = px.scatter(state_df[apr4_idx],x='SAH_cumexp',
                           y='cum_claims_to_emp',size='census2010pop',
                           size_max=35, hover_data=['state_name'],trendline="ols",                   
                           color='<b>Conf. C19 <br>per 1K p.c.</b>',
                           color_continuous_scale=px.colors.sequential.gray,)
fig_emp_scatt.update_traces(marker=dict(line=dict(width=.75,
                                        color='black')),
                            selector=dict(mode='markers'))

fig_emp_scatt.update_xaxes(title_text='Employment-Weighted Weeks of SAH Exposure',
                           tickfont=dict(color='black'),
                           title_font=dict(color='black'))
fig_emp_scatt.update_yaxes(title_text='Cumulative New Claims for UI (Mar. 15 - Apr. 4)',
                           tickfont=dict(color='black'),
                           title_font=dict(color='black'))

fig_emp_scatt.update_layout(
    plot_bgcolor='White'
)

fig_emp_scatt.show()

fig_emp_scatt.write_image("../../output/plots/UI-claims-scatter.pdf", 
                          width=900, height=500, scale=3)

In [8]:
states = [x for x in list(state_df.state_name.unique()) if x != '']

est_dict = {}
for state in states:
    results = smf.ols('cum_claims_to_emp ~ SAH_cumexp + confirm_to_1k ' + \
                      '+ middle_to_1k + wah + replacementrate',
                      data=state_df[apr4_idx].loc[state_df[apr4_idx]
                                             .state_name != state]).fit(cov_type='HC1')
    est_dict[state] = [results.params['SAH_cumexp'],results.HC1_se['SAH_cumexp']]

In [9]:
drop_state = pd.DataFrame.from_dict(est_dict).transpose()
drop_state.columns = ['beta','se']
drop_state.reset_index(inplace=True)
drop_state.rename(columns = {'index': 'State Dropped'},inplace=True)

drop_state['CI (90%)'] = drop_state['se']*norm.ppf(0.95)

fig_drop = px.scatter(drop_state, x="State Dropped", y="beta", error_y="CI (90%)")

fig_drop.update_xaxes(title_text='State Dropped from Benchmark Specification (90% CI)')
fig_drop.update_yaxes(title_text='Estimated Coefficient on SAH Exposure',rangemode="tozero")


fig_drop.show()

fig_drop.write_image("../../output/plots/drop_states_main.png", 
                     width=1200, height=600, scale=3)